# VOD 마트 데이터 전처리

- `name`과 `movie_id`의 중복 확인

<br><hr>

### 00. 기본 설정

In [2]:
import pandas as pd

# 경고 메시지 출력 X
import warnings
warnings.filterwarnings("ignore")

<br><hr>

### 01. 데이터 불러오기

In [16]:
mart = pd.read_csv('../data/영화_전처리ing.csv', encoding='utf8')
print(mart.shape)   # 5만 3874개

(53874, 19)


In [4]:
mart.columns

Index(['full_asset_id', 'release_year', 'genre', 'vote_count', 'vote_average',
       'runtime', 'popularity', 'poster_path', 'original_title',
       'original_language', 'backdrop_path', 'adult', 'release_date',
       'movie_id', 'orgnl_cntry', 'name', 'actr_disp', 'director', 'overview'],
      dtype='object')

In [5]:
mart.head(1)

,full_asset_id,release_year,genre,vote_count,vote_average,runtime,popularity,poster_path,original_title,original_language,backdrop_path,adult,release_date,movie_id,orgnl_cntry,name,actr_disp,director,overview
0,cjc|M4820686LSGE28658201,1977.0,"모험, 액션, SF",20724.0,8.204,121.0,118.411,/7XFfURIFCJxN1mfBg0SAjk5yGzg.jpg,Star Wars,en,/4qCqAdHcNKeAHcK8tJ8wNJZa9cx.jpg,False,1977-05-25,11,미국,스타워즈 4-새로운 희망,"마크 해밀,캐리 피셔,해리슨 포드,알렉 기네스,피터 커싱",조지 루카스,"공화국이 붕괴하고 제국이 수립된 뒤 20년, 제다이 기사단은 전멸하고 강력한 제국군..."


<br><hr>

### 02. name >> movie id >> ... groupby 확인

In [45]:
# 'name'으로 그룹화한 뒤, 각 그룹 내에서 고유한 movie_id의 개수를 계산
name_movieid_counts = mart.groupby('name')['movie_id'].nunique()
name_movieid_counts

name
#살아있다           1
#아이엠히어          1
#위왓치유           1
0.05%           1
0.0MHz          1
               ..
힐링메이트-무삭제       1
힘내세요 병헌씨        1
힘을 내요, 미스터 리    1
힘찬이는 자라서        1
힙노틱             1
Name: movie_id, Length: 13947, dtype: int64

In [50]:
# 고유한 movie_id가 2개 이상인 'name' 필터링
name_with_multiple_ids = name_movieid_counts[name_movieid_counts > 1]
print("같은 영화 이름(name)으로 여러 movie_id를 가진 경우:")
name_with_multiple_ids

같은 영화 이름(name)으로 여러 movie_id를 가진 경우:


name
가디언        2
가시         2
감자         3
겨울 이야기     2
고스트버스터즈    2
          ..
휴가         2
흙          2
희수         2
히든         3
히트맨        2
Name: movie_id, Length: 250, dtype: int64

In [51]:
# 해당 'name'에 대한 자세한 데이터프레임 추출
duplicates = mart[mart['name'].isin(name_with_multiple_ids.index)]

In [55]:
duplicates.to_csv('하나의name-두개이상movieid.csv', encoding='utf8', index=False)

In [53]:
# 중복 데이터 확인
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Movies with Multiple Movie IDs", dataframe=duplicates)

Movies with Multiple Movie IDs


In [54]:
mart.columns

Index(['full_asset_id', 'release_year', 'genre', 'vote_count', 'vote_average',
       'runtime', 'popularity', 'poster_path', 'original_title',
       'original_language', 'backdrop_path', 'adult', 'release_date',
       'movie_id', 'orgnl_cntry', 'name', 'actr_disp', 'director', 'overview'],
      dtype='object')

<br><hr>

### 02. asset 제외 모든 열의 값이 같은 행들 중복 제거

In [17]:
columns_to_check = ['release_year', 'genre', 'vote_count', 'vote_average', 'runtime',
                    'popularity', 'poster_path', 'original_title', 'original_language', 
                    'backdrop_path', 'adult', 'release_date', 'movie_id', 'orgnl_cntry', 
                    'name', 'actr_disp', 'director', 'overview'
]

In [39]:
# 중복 제거
unique_mart = mart[mart.duplicated(subset=columns_to_check, keep=False)]
unique_mart.shape   # 5만 3874개 >> 1만 6098개 (3만 7776개 삭제)

(49429, 19)

In [41]:
53874 - 49429

4445

In [43]:
unique_mart.head(2)

,full_asset_id,release_year,genre,vote_count,vote_average,runtime,popularity,poster_path,original_title,original_language,backdrop_path,adult,release_date,movie_id,orgnl_cntry,name,actr_disp,director,overview
0,cjc|M4820686LSGE28658201,1977.0,"모험, 액션, SF",20724.0,8.204,121.0,118.411,/7XFfURIFCJxN1mfBg0SAjk5yGzg.jpg,Star Wars,en,/4qCqAdHcNKeAHcK8tJ8wNJZa9cx.jpg,False,1977-05-25,11,미국,스타워즈 4-새로운 희망,"마크 해밀,캐리 피셔,해리슨 포드,알렉 기네스,피터 커싱",조지 루카스,"공화국이 붕괴하고 제국이 수립된 뒤 20년, 제다이 기사단은 전멸하고 강력한 제국군..."
1,cjc|M4820686LSGE82543601,1977.0,"모험, 액션, SF",20724.0,8.204,121.0,118.411,/7XFfURIFCJxN1mfBg0SAjk5yGzg.jpg,Star Wars,en,/4qCqAdHcNKeAHcK8tJ8wNJZa9cx.jpg,False,1977-05-25,11,미국,스타워즈 4-새로운 희망,"마크 해밀,캐리 피셔,해리슨 포드,알렉 기네스,피터 커싱",조지 루카스,"공화국이 붕괴하고 제국이 수립된 뒤 20년, 제다이 기사단은 전멸하고 강력한 제국군..."


In [31]:
# 중복된 값들만 저장 (제거된 행들)
group_counts = mart.groupby(columns_to_check).size()

duplicates = mart[mart.set_index(columns_to_check).index.isin(
    group_counts[group_counts > 1].index
)]
duplicates.shape

(41228, 19)

<br><hr>

### 03. name-movie_id 기준 중복 데이터 확인

- name-movie_id 값이 같지만 열에 다른 값이 있는 경우

In [8]:
duplicates = unique_mart.groupby(['name', 'movie_id']).filter(lambda x: len(x) > 1)
duplicates.shape

(3489, 19)

In [9]:
first = duplicates[:1001]
first.shape

(1001, 19)

In [10]:
second = duplicates[1001:2001]
second.shape

(1000, 19)

In [11]:
last = duplicates[2001:]
last.shape

(1488, 19)

In [57]:
first.to_csv('민주_중복.csv', encoding='utf8', index=False)
second.to_csv('상현_중복.csv', encoding='utf8', index=False)
last.to_csv('보리_중복.csv', encoding='utf8', index=False)

*이제 중복처리한 데이터 합친거랑 duplicates에 속하지 않는 데이터 합치면 완성!*

In [33]:
# 전처리한 데이터
df1 = pd.read_csv('보리_중복제거완정말.csv', encoding='utf8')
df2 = pd.read_csv('민주_중복_수정_최종.csv', encoding='utf8')
df3 = pd.read_csv('상현_중복 (완료).csv', encoding='utf8')

combined = [df1, df2, df3]

In [36]:
# 데이터프레임 병합
combined = pd.concat(combined, ignore_index=True)

# 결과 출력
print(combined.shape)

(3404, 19)


In [56]:
combined.to_csv('combined.csv', encoding='utf8', index=False)

In [29]:
# name과 movie_id가 중복되지 않은 정상적인 값
group_counts = unique_mart.groupby(['name', 'movie_id']).size()

# 1개 이하인 그룹 필터링
non_duplicates = unique_mart[unique_mart.set_index(['name', 'movie_id']).index.isin(
    group_counts[group_counts <= 1].index
)]

non_duplicates.shape

(12609, 19)

In [30]:
12609 + 3489

16098